In [1]:
import pandas as pd
import numpy as np 
import plotly.express as px
from tqdm import tqdm 
tqdm.pandas() 

C:\Users\romeo\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df = pd.read_csv('search_word.csv')
df2 = pd.read_csv('home_tv_reaction_log.csv') 

In [4]:
df['word_len']= df.apply(lambda x: len(str(x['srchwd'])), axis=1)
df.head() 

,imsi_no,host_site,srchwd,word_len
0,3DAFF61B191348505B0EBC8C5F45B820347915BD,googleapis.com,3472c807112edf6c697abfe277393614d1cf0c6daae9cc...,64
1,788D3D6BD0EC8316E9DB49535D449D6F51BCE703,google.com,탬,1
2,908AFC20E2CA5B6309E2203A4DF52353485BFAE7,ezadsystem.com,참치액,3
3,FC5C20BD5539DA443929EBFA9163994F121E40A6,halfclub.com,%ec%9a%94%ea%b0%80%eb%b8%8c%eb%9d%bc,36
4,4BE104D0C0A32CCD19FED382F8F4A6D1DD886FFA,googleapis.com,e472d9e963431875a5f7036ccb70d143d97fe22a6ef1fd...,64


In [5]:
import re

def clean_word(word):
    word = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"]', '',str(word)) #remove punctuation
    word = word.lower() #lower case
    word = re.sub(r'\s+', ' ', word) #remove extra space
    word = re.sub(r'<[^>]+>','',word) #remove Html tags
    word = re.sub(r"^\s+", '', word) #remove space from start
    #word = re.sub(r'\s+$', '', word) #remove space from the end
    return word

In [6]:
df = df[df.word_len < 20 ]
df['srchwd'] = df.srchwd.apply(clean_word)

r = re.compile('\D')
df['all_num'] = df.srchwd.apply(lambda x: r.search(x)) 
df = df.dropna()
df = df.drop(['all_num'], axis=1)
df.srchwd = df.srchwd.apply(lambda x: x+'/')
df.groupby('imsi_no')['srchwd'].sum()  
df 


,imsi_no,host_site,srchwd,word_len
1,788D3D6BD0EC8316E9DB49535D449D6F51BCE703,google.com,탬/,1
2,908AFC20E2CA5B6309E2203A4DF52353485BFAE7,ezadsystem.com,참치액/,3
5,135AEC6F05FECB8BBFBEB25ABA846AE343E91F27,findjob.co.kr,도장/,2
6,B59ED095E4D6F59FCAC2584478A05FB0B8C60B19,slrclub.com,스캐너/,3
7,B25A2422B339E610FE137DF884CBA0EE1F897486,auction.co.kr,로베르타 가방/,7
...,...,...,...,...
25053425,5886E97C896222EF547968A3321BEE6F503E881C,daum.net,공/,1
25053426,178CB2A3AB851483CB7D58F9127D0BF5CDA8233F,xn--vk1b209baa037j.kr,기아k9가격/,6
25053427,65797C73F7FD0717BB5792228EA31FDE8F667E1A,interpark.com,니트원피스/,5
25053428,91E7A75E0D23EE28741EC6BB8A2233816ECE86C0,samsungapps.com,토스/,2


In [7]:
tmp = df.groupby('imsi_no')['srchwd'].sum()
tmp = tmp.to_frame(name='keyword')
tmp['count'] = tmp.keyword.apply(lambda x: x.count('/'))
tmp.head()

,keyword,count
imsi_no,,
00000A827B976B36935DDF7A051847BB2A36548F,듀오락베이비/휴마시스 코로나자가키트/듀오락베이비/코로나자가키트/코로나자가키트/코로나...,8
00000E0458E19D321536C5F116A22D8ABB7BA4E2,무선청소기/무선청소기/무선청소기/무선청소기/기초화장품/무선청소기/무선청소기/울산교정...,16
00001C23602148AE349A7FBC5923BB2275935D85,계산기/,1
0000217D7CA13CAD17B0DC2B6F9DAA00014EDE9A,설강화 결말/,1
00002234A5010759526164E169DC54AB33FDFF9C,lg전자디오스/lg전자디오스/lg전자디오스/노트북/노트북/노트북/,6


In [80]:
tmp['keyword'] = tmp.keyword.apply(lambda x: x.split('/')[:-1])
tmp.head() 

,keyword,count
imsi_no,,
00000A827B976B36935DDF7A051847BB2A36548F,"[듀오락베이비, 휴마시스 코로나자가키트, 듀오락베이비, 코로나자가키트, 코로나자가키...",8
00000E0458E19D321536C5F116A22D8ABB7BA4E2,"[무선청소기, 무선청소기, 무선청소기, 무선청소기, 기초화장품, 무선청소기, 무선청...",16
00001C23602148AE349A7FBC5923BB2275935D85,[계산기],1
0000217D7CA13CAD17B0DC2B6F9DAA00014EDE9A,[설강화 결말],1
00002234A5010759526164E169DC54AB33FDFF9C,"[lg전자디오스, lg전자디오스, lg전자디오스, 노트북, 노트북, 노트북]",6


In [178]:
tmp_sample = tmp.head(50)
tmp_sample = tmp_sample.reset_index()
tmp_sample

,imsi_no,keyword,count
0,00000A827B976B36935DDF7A051847BB2A36548F,"[듀오락베이비, 휴마시스 코로나자가키트, 듀오락베이비, 코로나자가키트, 코로나자가키...",8
1,00000E0458E19D321536C5F116A22D8ABB7BA4E2,"[무선청소기, 무선청소기, 무선청소기, 무선청소기, 기초화장품, 무선청소기, 무선청...",16
2,00001C23602148AE349A7FBC5923BB2275935D85,[계산기],1
3,0000217D7CA13CAD17B0DC2B6F9DAA00014EDE9A,[설강화 결말],1
4,00002234A5010759526164E169DC54AB33FDFF9C,"[lg전자디오스, lg전자디오스, lg전자디오스, 노트북, 노트북, 노트북]",6
5,0000366524FE3E78FC6D994A7670EC5406F94B3A,"[종근당락토핏, 종근당락토핏]",2
6,0000437748A418065D4D1B49A702CB07D38A305E,[korean girl],1
7,000059088E55A3B7A6FA0BD93980ACC326327E17,"[부천가구단지, 부천가구단지, 부천가구단지, 부천가구단지, 콘센트]",5
8,00005B7253AF369BB80B88DE22369F8658B74A9B,"[더프트앤도프트, 클럽의상, 더프트앤도프트, 더프트앤도프트, 더프트앤도프트, 클럽의상]",6
9,00005CD43CBB137E1AF9A8A55414431A73144D48,"[우일요, 글라스락]",2


In [16]:
df_key = df2.copy()

In [184]:
df_key.head()

,base_dt,cust_no,fmil_id,svc_id,chnl_nm,pgm_nm,gnre_nm,catg_cd,lvl1_hshp_prod_catg_nm,lvl2_hshp_prod_catg_nm,...,wtch_strt_ltr_ract_tm_scnt,wtch_end_ltr_ract_tm_scnt,ract_kd_nm,stlm_yn,wtch_end_ltr_yn,bdp_load_user_id,bdp_load_dttm,p_basis_yyyy,p_basis_mm,p_basis_dd
0,20220124,02978619C7F0205A42E901C0118E2B7BD9AA8EC4,H9414801,674,롯데홈쇼핑,"[심영순, 찜굴비]",설마음한상,4668.0,식품,축산,...,420,-110,1,0.0,1,HP_L2FWT_HSE_HSP_WTCH_REACT,2022-02-11 14:23:52,2022,1,24
1,20220124,040488C93782D664B645B25ADA2C34D669928294,H5089765,737,공영쇼핑,[감자옹심이],가공농산,NaN,NaN,NaN,...,1592,-425,1,0.0,1,HP_L2FWT_HSE_HSP_WTCH_REACT,2022-02-11 14:23:52,2022,1,24
2,20220124,072F718733103B3BCF76DACFF13C6F08F3E9194C,H7402626,811,K쇼핑,"[다움, 여주환]",식품,1489.0,식품,건강식품,...,117,-1437,1,0.0,1,HP_L2FWT_HSE_HSP_WTCH_REACT,2022-02-11 14:23:52,2022,1,24
3,20220124,093CE798150B4F527662999599BB1C6EC54104FF,H7652877,737,공영쇼핑,[제주은갈치옥돔세트],신선수산,4582.0,식품,수산,...,400,136,1,0.0,0,HP_L2FWT_HSE_HSP_WTCH_REACT,2022-02-11 14:23:52,2022,1,24
4,20220124,0A53AFD988587918B0011E6239B2B09C05F70EA8,H2755802,782,쇼핑엔티,"[흥국생명, 여성암보험]",금융,976.0,생활/건강,무형서비스,...,13,-666,1,0.0,1,HP_L2FWT_HSE_HSP_WTCH_REACT,2022-02-11 14:23:52,2022,1,24


In [18]:
df_key['pgm_nm'] = df_key.pgm_nm.apply(lambda x: x.split())

In [21]:
item_list = df_key['pgm_nm']

In [35]:
item_list_full = []
for i in item_list:
    item_list_full.extend(i)
len(item_list_full)        

4366531

In [41]:
item_list_full_set = list(set(item_list_full))

In [105]:
item_list_full_set

['안마기',
 '진비단',
 '쥬비스다이어트',
 '선진모피',
 '다이얼락',
 'LG모니터',
 '라인',
 '반건조',
 '기초세트',
 '재무설계',
 '유니케어세정제<재>',
 '파뷔에글래드팬세트,eTV',
 '페이셜크림)',
 '까사맘스텐용기',
 '흑당고',
 '할리마리프린트점퍼',
 '붙박이장',
 '수바이오순댓국(상생)',
 '마이셰프',
 '루컴즈',
 '셀티바',
 '고객만족정보세상',
 '이어폰',
 '마데카소사이드',
 '리앤케이기초',
 '티에스',
 '소담찬원데이-골프채',
 '혼방',
 '전복',
 '르까프',
 '테팔차탕기',
 '쪽갈비',
 '시즌2',
 '나인웨스트',
 '한춘섭된장시래기<재>',
 '에넥스',
 '나잘비염치료기',
 '프로젝트]비타바움',
 '플라워',
 '튜브',
 '프로그램',
 '1등급한우구이세트',
 '소프트',
 '램프쿡냄비',
 '엠아잠',
 '테팔청소기',
 '잡화',
 '수리치영양떡',
 '로잉머신',
 '스키니랩',
 '마이크로',
 '[렌탈]대명',
 '에너제틱',
 '밍크카페트',
 '빅마마사태맑은국',
 '무니',
 '청송',
 '크린조이',
 '쌍화배도라지',
 '마르조이지웨어21',
 '페이즐리',
 '수도여과기',
 '수리',
 '바네사리찌기모블라우스2',
 '마르조',
 '특집',
 '선진모피밍크모자머플러21,선진모피밍크베스트21',
 '타월',
 '생활꿀팁2',
 '캐나다',
 '새벽집',
 '셔츠/아트핏',
 '손질새우',
 '모션데스크+모니터암',
 '패딩자켓',
 '심야',
 '바다포차오징어',
 '그렉노먼',
 '닥터플랜츠',
 '보랄',
 '붕어빵',
 '라자',
 '비접촉',
 '현대정수기',
 '투어',
 'V6',
 '매직캔쓰레기통',
 '핸드백(초특가)',
 '레더',
 '세상',
 '닥터큐먼콜라겐팩',
 '올바이오',
 '보오글네모팬<재>',
 '경량패딩',
 'TV렌탈',
 '캐치웰청소기',
 '국내산돌문어(경양)',
 '가스레인지',
 '마스터',
 '제니

In [179]:
sim_result = pd.DataFrame(columns=['id','search','item_key','similarity'])
id_list = []
search_list = []
item_key_list = []
similarity_list = []

In [97]:
def word2vec(word):
    from collections import Counter
    from math import sqrt

    # count the characters in word
    cw = Counter(word)
    # precomputes a set of the different characters
    sw = set(cw)
    # precomputes the "length" of the word vector
    lw = sqrt(sum(c*c for c in cw.values()))

    # return a tuple
    return cw, sw, lw

def cosdis(v1, v2):
    # which characters are common to the two words?
    common = v1[1].intersection(v2[1])
    # by definition of cosine distance we have
    return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]

In [180]:
import itertools
#list_A = ['lg전자디오스','노트북','계산기','기초화장품','무선청소기']
list_B = item_list_full_set

threshold = 0.8     # if needed
for i in tmp_sample['imsi_no'] :
    list1 = list(tmp_sample[tmp_sample['imsi_no']==i]['keyword'])
    merged = list(itertools.chain(*list1))
    #print(i)
    for key in merged:
        #print(key)
        for word in list_B:
            try:
                # print(key)
                # print(word)
                res = cosdis(word2vec(word), word2vec(key))
                # print(res)
                # print("The cosine similarity between : {} and : {} is: {}".format(word, key, res*100))
                if res > threshold:
                    #print("Found a id : {} word with cosine distance > 80 : {} with original word: {}".format(i,word, key))
                    id_list.append(i)
                    search_list.append(key)
                    item_key_list.append(word)
                    similarity_list.append(res)
            except IndexError:
                pass

In [181]:
print(len(id_list))
print(len(search_list))
print(len(item_key_list))

86
86
86


In [182]:
sim_result['id'] = id_list
sim_result['search'] = search_list
sim_result['item_key'] = item_key_list
sim_result['similarity'] = similarity_list
sim_result['count'] = 0

In [183]:
group_sim = sim_result.groupby(['id','search','item_key','similarity'])
group_sim.count().reset_index()

,id,search,item_key,similarity,count
0,00000E0458E19D321536C5F116A22D8ABB7BA4E2,무선청소기,LG무선청소기,0.845154,10
1,00000E0458E19D321536C5F116A22D8ABB7BA4E2,무선청소기,무선진공청소기,0.845154,10
2,00000E0458E19D321536C5F116A22D8ABB7BA4E2,무선청소기,무선청소기,1.000000,10
3,00002234A5010759526164E169DC54AB33FDFF9C,노트북,노트북,1.000000,3
4,00005CD43CBB137E1AF9A8A55414431A73144D48,글라스락,글라스락,1.000000,1
5,00006CEE1AA73BFE58EDAF7C4DAEB7850051E842,양말,양말,1.000000,10
6,00006CEE1AA73BFE58EDAF7C4DAEB7850051E842,유두,두레유,0.816497,10
7,00006CEE1AA73BFE58EDAF7C4DAEB7850051E842,유두,두유,1.000000,10
8,00006CEE1AA73BFE58EDAF7C4DAEB7850051E842,유두,진두유,0.816497,10
9,000167B9947CE8444FDEBD396EAD71A4EE834082,당근,종근당,0.816497,1


In [ ]:
group_sim.join(df_key.set_index('MODEL')['MAKE'], on='MODEL')

In [ ]:
group_sim['category_lv2'] = np.where(group_sim[]df_key['pgm_nm'](lvl2_hshp_prod_catg_nm)

In [14]:
from gensim import models

ko_model = models.fasttext.load_facebook_model('../cc.ko.300.bin.gz')  


In [24]:
ko_model.wv.word_vec('갈비찜')  


array([-3.84397134e-02, -1.56466700e-02, -2.06001475e-02, -2.88120583e-02,
        2.01578066e-02,  8.82993639e-02, -2.93946508e-02,  1.18517146e-01,
        8.37789103e-02,  4.94580204e-03, -8.64327550e-02,  1.13190161e-02,
       -2.27398463e-02, -6.69045299e-02, -2.42107809e-02,  2.18320414e-02,
        1.85002089e-02,  4.39051203e-02, -5.80373257e-02, -2.70043872e-02,
        2.83007864e-02,  1.77512504e-03,  3.63254957e-02, -6.01904951e-02,
       -6.55479580e-02,  3.70736003e-01, -5.10404184e-02, -1.57314446e-02,
       -7.78125748e-02,  6.37522638e-02, -2.02954914e-02, -1.60282273e-02,
        4.82784845e-02, -2.95918025e-02, -1.00528277e-01,  7.81376511e-02,
        3.36652286e-02,  5.88492081e-02,  1.22010484e-01,  3.72566953e-02,
        7.48469820e-03, -7.79916346e-03,  6.96974471e-02,  4.12570126e-02,
        1.52645549e-02,  1.13347501e-01, -6.95192069e-02,  3.88332121e-02,
        1.07074969e-01, -1.03934489e-01,  5.79874739e-02,  1.94211435e-02,
       -3.87453027e-02,  

In [23]:
ko_model.wv.most_similar('갈비찜')

[('두부조림', 0.7227581739425659),
 ('떡갈비', 0.7181905508041382),
 ('간장게장과', 0.7129815220832825),
 ('낙지볶음', 0.6988350749015808),
 ('갈비탕', 0.6921705603599548),
 ('더덕구이', 0.6907923221588135),
 ('계란찜', 0.6906286478042603),
 ('족발과', 0.6900067925453186),
 ('돼지갈비찜', 0.6842390298843384),
 ('만두전골', 0.6841294169425964)]

In [25]:
ko_model.wv.similarity('굴비','고등어')

0.30286112

In [26]:
ko_model.wv.similarity('굴비','해수욕장')

0.10835029

In [27]:
ko_model.wv.similarity('삼성','엘지')

0.5731262

In [28]:
ko_model.wv.similarity('삼성','팥빙수')

0.13735418

In [31]:
ko_model.wv.similarity('삼성','노트북')

0.32805696

In [32]:
ko_model.wv.similarity('엘지','노트북') 

0.24805987

In [30]:
ko_model.wv.most_similar('삼성전자')

[('삼성전자의', 0.7385889291763306),
 ('메모리사업부', 0.6666884422302246),
 ('반도체총괄', 0.6560724377632141),
 ('삼성전자가', 0.6429705619812012),
 ('DS부문', 0.64190274477005),
 ('쿠바서', 0.6372543573379517),
 ('화성캠퍼스', 0.6306094527244568),
 ('반도체사업부', 0.6299896240234375),
 ('화성사업장', 0.6291687488555908),
 ('삼성전자는', 0.6287757754325867)]